# Table of contents
1. [Exploratory Factor Analysis (EFA) Results](#EFA)
    1. [Survey EFA](#survey-EFA)
    2. [Task EFA](#task-EFA)
    3. [Outcome EFA](#outcome-EFA)
2. [EFA Robustness](#EFA-robustness)
    1. [Survey EFA robustness](#survey-EFA-robustness)
    2. [Task EFA robustness](#task-EFA-robustness)
3. [Hierarchical Clustering (HC)](#HCA)
    1. [Survey HC clusters](#survey-HCA)
    2. [Task HC clusters](#task-HCA)
4. [HC Robustness](#HCA-robustness)
    1. [Survey HC clusters](#survey-HCA-robustness)
        1. [Adjusted mutual info results](#survey-HCA-robustness-AMI)
        2. [Consensus Clustering Distances](#survey-HCA-robustness-close)
        3. [Original Clusters consensus distances](#survey-HCA-robustness-cluster)
    2. [Task HC clusters](#task-HCA-robustness)
        1. [Adjusted mutual info results](#task-HCA-robustness-AMI)
        2. [Consensus Clustering Distances](#task-HCA-robustness-close)
        3. [Original Clusters consensus distances](#task-HCA-robustness-cluster)

**Supplemental Figures for _Uncovering mental structure through data-driven ontology discovery_**

This notebook contains:
- Loading matrices for task, survey and outcome exploratory factor analyses
- Robustness analysis for task and survey factor analyses
- All clusters from the hierarchical clusting (Figures 4 and 5 from the main paper) separately plotted out with DVs labeled
- Robustness analyses for hierarchical clustering, comparing the clustering reported in the paper to perturbed data and a consensus clustering approach

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from os import path
import pandas as pd
import pickle
import seaborn as sns
pd.set_option('max_rows', 200)

from dimensional_structure import DA_plots
from dimensional_structure.EFA_cluster_robustness import ConsensusCluster
from dimensional_structure.EFA_plots import plot_heatmap_factors, plot_bar_factors, plot_factor_correlation
from dimensional_structure.HCA_plots import plot_subbranches, plot_results_dendrogram
from dimensional_structure.notebook_utils import (plot_factor_df, plot_EFA_robustness, 
                                                  plot_bootstrap_results,
                                                  plot_HCA_AMI,
                                                  display_closest_DVs,
                                                  display_cluster_DVs)
from selfregulation.utils.utils import get_recent_dataset
from selfregulation.utils.result_utils import load_results

In [ ]:
%matplotlib inline
dataset = get_recent_dataset()
results = load_results(datafile=dataset)

# Exploratory Factor Analysis Results <a class="anchor" id="EFA"></a>

Below are the loading matrices for the exploratory factor analysis (EFA) solutions for surveys, tasks, and the outcome measures.

### Survey Exploratory Factor Analysis Loadings<a class="anchor" id="survey-EFA"></a>

12 factors were determined using a BIC criteria for exploratory factor analysis. The 64 survey DVs are grouped and ordered based on the largest (absolute) factor loading for that DV. **Hover over the cells to see the exact value**

In [ ]:
fig = plt.figure(figsize=(12, 3))
ax1 = fig.add_axes([0.05, 0.80, 0.9, 0.15])
cmap = sns.diverging_palette(220,15,n=20, as_cmap=True)
cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap,
                                orientation='horizontal')
cb1.set_ticks([0,.5,1])
cb1.set_ticklabels(['-1','0', '1'])
cb1.set_label('Color Bar for Factor Loading Plots')

In [ ]:
survey_results = results['survey']
plot_factor_df(survey_results.EFA)

### Task Exploratory Factor Analysis Loadings<a class="anchor" id="task-EFA"></a>

5 factors were determined using a BIC criteria for exploratory factor analysis. The 129 survey DVs are grouped and ordered based on the largest (absolute) factor loading for that DV. Dotted lines indicate separate groups derived from this criteria, and are used for visualization purposes only.

In [ ]:
task_results = results['task']
plot_factor_df(task_results.EFA)

### Outcome Exploratory Factor Analysis Loadings<a class="anchor" id="outcome-EFA"></a>

9 factors were determined using a BIC criteria for exploratory factor analysis. The 55 target measures are grouped and ordered based on the largest (absolute) factor loading for that target measure. Dotted lines indicate separate groups derived from this criteria, and are used for visualization purposes only.

In [ ]:
plot_factor_df(task_results.DA)

# Factor Robustness Analyses<a class="anchor" id="EFA-robustness"></a>

Factor robustness proceeded in two ways:

**(1)** By dropping one measure at a time, recalculating the survey factor solution (with the same number of factors as the full sample) and correlating the new factor loadings with the original factor loadings. This correlation is calculated on all DVs excepting those dropped out because of the dropped measure. Tables for these correlations are shown with values below .9 highlighted in red

**(2)** By using a bootstrap procedure (see fa.sapa), which creates confidence intervals for each loading. 95% confidence intervals are plotted as bar plots for each loading

## Survey Robustness<a class="anchor" id="survey-EFA-robustness"></a>

### Measurement Drop

Individual surveys sometimes had large effects on the factor structure, likely because of spare measurement of highly discriminat psychological constructs. All dimensions that are affected by a measure drop out (correlation pre-post drop out of factor loadings < .9) are highlighted. Worse deviation are colored darker.

In [ ]:
f = path.join(results['survey'].get_output_dir(), 'EFAdrop_robustness.pkl')
survey_EFA_robustness = pickle.load(open(f, 'rb'))
plot_EFA_robustness(survey_EFA_robustness)

### Bootstrap

It's clear that loadings are robust to the particulars of the sample. The standard deviation of the loadings are very small relative to the mean loadings.

In [ ]:
plot_bootstrap_results(survey_results.EFA.get_boot_stats())

## Task Robustness<a class="anchor" id="task-EFA-robustness"></a>

### Measurement Drop

Task factors are more robust to dropping out particular measures, likely due to the greater overlap in the psychological constructs measured by individual tasks

In [ ]:
f = path.join(results['task'].get_output_dir(), 'EFAdrop_robustness.pkl')
task_EFA_robustness = pickle.load(open(f, 'rb'))
plot_EFA_robustness(task_EFA_robustness)

### Bootstrap

It's clear that loadings are robust to the particulars of the sample. The standard deviation of the loadings are very small relative to the mean loadings.

In [ ]:
plot_bootstrap_results(task_results.EFA.get_boot_stats())

# Hierarchical Clustering<a class="anchor" id="HCA"></a>

Hierarchical clustering was used to order dependent variables based on the similarity of their loading vectors. This resulted in a dendrogram, which was subset into clusters using the DynamicTreeCut algorithm. These clusters are separately plotted below, allowing the constituent DVs to be read.

### Survey Clusters<a class="anchor" id="survey-HCA"></a>

Below is the survey dendrogram (reproduced from the main manuscript). Following are the 12 clusters. separately plotted. The fourth and fifth clusters, referenced in the main text, together reflect canonical components of "self-control".

In [ ]:
_ = plot_results_dendrogram(survey_results, size=20, drop_list=[0,2,4,6,8,10])

In [ ]:
plot_subbranches(survey_results, size=6)

### Task Clusters<a class="anchor" id="task-HCA"></a>

Below is the task dendrogram (reproduced from the main manuscript). Following are the 15 of the 16 clusers separately plotted. One cluster composed of a single DV (towards the center of the dendrogram) is not plotted. The single DV is the "Positive Learning Bias" from the "Probabilistic Selection Task"

In [ ]:
_ = plot_results_dendrogram(task_results, size=20, drop_list=[1,5,9,12,15], double_drop_list=[2,4,6,8,14])

In [ ]:
plot_subbranches(task_results, size=6)

# Cluster Robustness Analyses<a class="anchor" id="HCA-robustness"></a>

Clustering was repeated 5000 times with perturbed loading matrices. First, each simulation used only 80% of the total DVs. Second, each loading value was sampling from a gaussian with mean and standard deviation determined by the EFA bootstrap method. Finally, the same clustering approach described in the main paper is applied (hierarchical clustering followed by dynamicTreeCut).

Each simulation produces a co-occurence matrix: 1 if two variables were clustered together, and 0 otherwise. Averaged over simulations this produces a more robust estimate of the probability two DVs should be clustered (ranging from 0% to 100%). 1-P(clustering together) is then used as a distance matrix to perform one final clustering which creates a "consensus cluster".

We provide plots to show three aspects of the clustering:
- The AMI values when comparing the original clustering to simulations or the consensus clustering
- The list of DVs that are most often clustered with each DV (i.e., closest DVs to each DV according to the consensus clustering
- The closest variables contained in the same cluster according to the original clustering solution above

## Survey Cluster Robustness <a class="anchor" id="survey-HCA"></a>

### AMI values<a class="anchor" id="survey-HCA-robustness-AMI"></a>

In [ ]:
f = path.join(results['survey'].get_output_dir(), 'cluster_robustness.pkl')
survey_cluster_robustness = pickle.load(open(f, 'rb'))
survey_consensus = survey_cluster_robustness['consensusClust']
plot_HCA_AMI(survey_consensus)

### Closest values according to consensus clustering<a class="anchor" id="survey-HCA-robustness-close"></a>

Hover over each cell to bold the DV. Columns are organized by first closest DV to eighth closest DV. Each cell contains a DV name and the percent of simulations it was clustered with the DV defining the row.

In [ ]:
display_closest_DVs(survey_consensus, n_closest=8)

### Consensus clustering distances of variables within the same cluster according to original cluster<a class="anchor" id="survey-HCA-robustness-cluster"></a>

Hover over each cell to bold the DV. Columns are organized by closeness of DVs within cluster.

For instance, the first variable BIS11 Survey: Attentional is part of a cluster with 4 variables called "Mindfulness". The first two closest DVs were always clustered with BIS11: Attentional (across the 5000 simulations), while the furthest  DV was only clustered with it 93% of the time.

Each cell contains a DV name and the percent of simulations it was clustered with the DV defining the row. Rows with more "red" indicate that the DV in that row was closer to all other DVs within the cluster.

In [ ]:
display_cluster_DVs(survey_consensus, survey_results)

## Task Cluster Robustness <a class="anchor" id="task-HCA-robustness"></a>

### Adjusted mutual information<a class="anchor" id="task-HCA-robustness-AMI"></a>

Below is the distribution of adjusted mutual information scores comparing the original task clustering reported in the main paper to simulated clusters and the consensus cluster.

In [ ]:
f = path.join(results['task'].get_output_dir(), 'cluster_robustness.pkl')
task_cluster_robustness = pickle.load(open(f, 'rb'))
task_consensus = task_cluster_robustness['consensusClust']
plot_HCA_AMI(task_consensus)

### Closest values according to consensus clustering<a class="anchor" id="task-HCA-robustness-close"></a>

Hover over each cell to bold the DV. Columns are organized by first closest DV to eighth closest DV. Each cell contains a DV name and the percent of simulations it was clustered with the DV defining the row.

In [ ]:
display_closest_DVs(task_consensus)

### Consensus clustering distances of variables within the same cluster according to original cluster<a class="anchor" id="task-HCA-robustness-cluster"></a>

Hover over each cell to bold the DV. Columns are organized by closeness of DVs within cluster.

For instance, the first variable, the Adaptive N-back drift rate, is part of a cluster with 9 DVs called "Conflict Processing". The closest DV was clustered with the adaptive n-back drift rate 79%, while the furthest DV in the cluster was associated 35% of the time. 

Each cell contains a DV name and the percent of simulations it was clustered with the DV defining the row.Rows with more "red" indicate that the DV in that row was closer to all other DVs within the cluster.

In [ ]:
display_cluster_DVs(task_consensus, task_results)